# EasyCV多目标跟踪

本文将介绍如何使用EasyCV快速使用FCOS检测器和ReID进行多目标跟踪的训练、推理

## 安装依赖包

注: 在PAI-DSW docker中无需安装相关依赖，可跳过此部分 在本地notebook环境中执行


1、 首先，安装pytorch和对应版本的torchvision，支持Pytorch1.5.1以上版本

In [ ]:
# install pytorch and torch vision
! conda install --yes pytorch==1.10.0 torchvision==0.11.0 -c pytorch

2、 获取torch和cuda版本，安装对应版本的mmcv和nvidia-dali

In [ ]:
import torch
import os
os.environ['CUDA']='cu' + torch.version.cuda.replace('.', '')
os.environ['Torch']='torch'+torch.version.__version__.replace('+PAI', '')
!echo $CUDA
! echo $Torch

In [ ]:
# install some python deps
! pip install mmcv-full==1.4.4 -f https://download.openmmlab.com/mmcv/dist/${CUDA}/${Torch}/index.html
! pip install http://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/third_party/nvidia_dali_cuda100-0.25.0-1535750-py3-none-manylinux2014_x86_64.whl

3、  安装EasyCV算法包

In [ ]:
pip install pai-easycv

## 简单验证

In [ ]:
from easycv.apis import *

## 运行环境要求

PAI-Pytorch 1.7/1.8镜像， GPU机型 P100 or V100， 内存 32G

### 数据准备
先通过docs/source/data_hub.md下载crowdhuman、mot20和Market1501三个数据集，然后使用tools/prepare_data/目录下的crowdhuman2coco.py、mot2coco.py和prepare_market1501.py文件将三个数据集转化成coco格式

data/coco格式如下

```shell
data/coco/
├── annotations
│   ├── train.json
│   └── val.json
├── train
│   ├── 000000005802.jpg
│   ├── 000000060623.jpg
│   ├── 000000086408.jpg
│   ├── 000000118113.jpg
│   ├── 000000184613.jpg
│   ├── 000000193271.jpg
│   ├── 000000222564.jpg
│       ...
│   └── 000000574769.jpg
└── val
    ├── 000000006818.jpg
    ├── 000000017627.jpg
    ├── 000000037777.jpg
    ├── 000000087038.jpg
    ├── 000000174482.jpg
    ├── 000000181666.jpg
    ├── 000000184791.jpg
    ├── 000000252219.jpg
         ...
    └── 000000522713.jpg
```

### 训练FCOS检测模型
下载训练配置文件

In [ ]:
! rm -rf fcos_r50_torch_1x_mot20_crowdhuman.py
!wget https://raw.githubusercontent.com/alibaba/EasyCV/master/configs/detection/fcos/fcos_r50_torch_1x_mot20_crowdhuman.py

使用单卡gpu进行训练和验证集评估

In [ ]:
! python -m easycv.tools.train  fcos_r50_torch_1x_mot20_crowdhuman.py --work_dir work_dirs/mot/fcos/

### 训练ReID模型
下载训练配置文件

In [ ]:
! rm -rf market1501_resnet50_jpg.py
!wget https://raw.githubusercontent.com/alibaba/EasyCV/master/configs/classification/imagenet/resnet/market1501_resnet50_jpg.py

In [ ]:
! python -m easycv.tools.train  market1501_resnet50_jpg.py --work_dir work_dirs/mot/reid/

In [ ]:
评估ReID模型(checkpoint路径替换成work_dirs/mot/reid/epoch_60_export.pt)

In [ ]:
! python -m unittest tests/predictors/test_reid_predictor.py

In [ ]:
使用预先训练好的reid模型

In [ ]:
! python easycv/thirdparty/reid/inference.py --checkpoint https://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/tracking/reid_r50_epoch_60_export.pt

### 预测
下载测试视频

In [ ]:
! wget https://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/tracking/MOT20-01.zip && unzip MOT20-01.zip

In [ ]:
单镜头MOT

In [ ]:
! python easycv/thirdparty/mot/demo_mot.py --input MOT20-01/img1 --output work_dirs/mot/result.mp4 --checkpoint work_dirs/mot/fcos/epoch_12_export.pt --fps 24

In [ ]:
使用预先训练好的fcos模型

In [ ]:
! python easycv/thirdparty/mot/demo_mot.py --input MOT20-01/img1 --output work_dirs/mot/result.mp4 --checkpoint https://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/tracking/fcos_r50_epoch_12_export.pt --fps 24

In [ ]:
跨镜头MOT

In [ ]:
! python easycv/thirdparty/mot/demo_mtmct.py --det_checkpoint work_dirs/mot/fcos/epoch_12_export.pt --reid_checkpoint work_dirs/mot/reid/epoch_60_export.pt --input mtmct-demo/ --output work_dirs/mot/ --save_images --use_folder

In [ ]:
使用预先训练好的fcos和reid模型

In [ ]:
! python easycv/thirdparty/mot/demo_mtmct.py --det_checkpoint https://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/tracking/fcos_r50_epoch_12_export.pt --reid_checkpoint https://pai-vision-data-hz.oss-cn-zhangjiakou.aliyuncs.com/data/tracking/reid_r50_epoch_60_export.pt --input mtmct-demo/ --output work_dirs/mot/ --save_images --use_folder